#KNN BASED RECOMMENDATION SYSTEM

Movie Recommendation

In [ ]:
pip install fuzzywuzzy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
from scipy.sparse import csr_matrix
from fuzzywuzzy import process
from sklearn.neighbors import NearestNeighbors
import numpy as np

/usr/local/lib/python3.10/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [ ]:
df_movies = pd.read_csv(
    movies_filename,
    usecols=['movieId', 'title'],
    dtype={'movieId': 'int32', 'title': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    usecols=['userId', 'movieId', 'rating'],
    dtype={'userId': 'int32', 'movieId': 'int32', 'rating': 'float32'})    

In [ ]:
from scipy.sparse import csr_matrix
# pivot ratings into movie features
df_movie_features = df_ratings.pivot(
    index='movieId',
    columns='userId',
    values='rating'
).fillna(0)
# convert dataframe of movie features to scipy sparse matrix
mat_movie_features = csr_matrix(df_movie_features.values)

In [ ]:
df_movie_features.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,0.0,0.0,4.0,0.0,4.5,0.0,0.0,0.0,...,4.0,0.0,4.0,3.0,4.0,2.5,4.0,2.5,3.0,5.0
2,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,0.0,0.0,...,0.0,4.0,0.0,5.0,3.5,0.0,0.0,2.0,0.0,0.0
3,4.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
from sklearn.neighbors import NearestNeighbors
model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=20, n_jobs=-1)

In [ ]:
movies_users=df_ratings.pivot(index='movieId', columns='userId',values='rating').fillna(0)
mat_movies_users=csr_matrix(movies_users.values)

In [ ]:
mat_movies_users

<9724x610 sparse matrix of type '<class 'numpy.float32'>'
	with 100836 stored elements in Compressed Sparse Row format>

In [ ]:
model_knn= NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=20)

In [ ]:
model_knn.fit(mat_movies_users)

NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=20)

In [ ]:
# Recommender(movie_name) => List of Movies recommended

def recommender(movie_name, data,model, n_recommendations ):
    model.fit(data)
    idx=process.extractOne(movie_name, df_movies['title'])[2]
    print('Movie Selected: ',df_movies['title'][idx], 'Index: ',idx)
    print('Searching for recommendations.....')
    distances, indices=model.kneighbors(data[idx], n_neighbors=n_recommendations)
    for i in indices:
        print(df_movies['title'][i].where(i!=idx))
    
recommender('iron man', mat_movies_users, model_knn,20)

Movie Selected:  Iron Man (2008) Index:  6743
Searching for recommendations.....
6743                                            NaN
7197                                  Garage (2007)
7195                        Merry Madagascar (2009)
7354                             A-Team, The (2010)
6726                         Superhero Movie (2008)
7137                         Thirst (Bakjwi) (2009)
7026                                 Scorpio (1973)
7571                                 Win Win (2011)
3880                  Look Who's Talking Now (1993)
6388    After the Wedding (Efter brylluppet) (2006)
7601                       Idiots and Angels (2008)
6755                Nina's Heavenly Delights (2006)
7022                                   Earth (2007)
7338                          Blue Valentine (2010)
4421                         What's Up, Doc? (1972)
6195                             Silent Hill (2006)
5885                                   Crash (2004)
3740                              M

Rating prediction

In [ ]:
!pip install surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 10.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3095463 sha256=1b91945f2db8e16e71185ade9fefcef09b67b71ad4b5e25f273a25b75b19d6fd
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [ ]:
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise.prediction_algorithms import KNNBaseline, KNNBasic, KNNWithMeans, KNNWithZScore
from surprise import accuracy
from surprise.model_selection import train_test_split

In [ ]:
def convert_traintest_dataframe_forsurprise(training_dataframe, testing_dataframe):
    reader = Reader(rating_scale=(0, 5))
    trainset = Dataset.load_from_df(training_dataframe[['userId', 'movieId', 'rating']], reader)
    testset = Dataset.load_from_df(testing_dataframe[['userId', 'movieId', 'rating']], reader)
    trainset = trainset.construct_trainset(trainset.raw_ratings)
    testset = testset.construct_testset(testset.raw_ratings)
    return trainset, testset

In [ ]:
def recommendation(algo, trainset, testset):
  # Train the algorithm on the trainset, and predict ratings for the testset
  algo.fit(trainset)

  # Predictions on testing set
  test_predictions = algo.test(testset)
  test_rmse = accuracy.rmse(test_predictions)
  test_mae = accuracy.mae(test_predictions)
  
  return test_rmse, test_mae, test_predictions

In [ ]:
#KNNBaseline
sim_options = {'name': 'pearson',
               'user_based': False  # compute  similarities between items
               }
algo = KNNBaseline(k=20,sim_options=sim_options)
test_knn_rmse, test_knn_mae, test_knn_pred = recommendation(algo, trainset, testset)
     

Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 0.9016
MAE:  0.7079


In [ ]:
#KNNBaseline
sim_options = {'name': 'pearson',
              'user_based': True  # compute similarities between users
               }
algo = KNNBaseline(k=20,sim_options=sim_options)
test_knn_rmse, test_knn_mae, test_knn_pred = recommendation(algo, trainset, testset)

Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 0.9087
MAE:  0.7162


In [ ]:
test_knn_pred

[Prediction(uid='4048', iid='1839', r_ui=3.0, est=2.6712590209071063, details={'actual_k': 20, 'was_impossible': False}),
 Prediction(uid='1181', iid='2871', r_ui=3.0, est=3.189054489428745, details={'actual_k': 20, 'was_impossible': False}),
 Prediction(uid='451', iid='2612', r_ui=4.0, est=4.760733805989169, details={'actual_k': 20, 'was_impossible': False}),
 Prediction(uid='673', iid='2818', r_ui=1.0, est=1, details={'actual_k': 20, 'was_impossible': False}),
 Prediction(uid='4524', iid='2699', r_ui=3.0, est=3.1638907683805346, details={'actual_k': 20, 'was_impossible': False}),
 Prediction(uid='2472', iid='3591', r_ui=3.0, est=3.1057830063881178, details={'actual_k': 20, 'was_impossible': False}),
 Prediction(uid='5208', iid='196', r_ui=4.0, est=3.267851282467091, details={'actual_k': 20, 'was_impossible': False}),
 Prediction(uid='1726', iid='3441', r_ui=2.0, est=2.5531203996292664, details={'actual_k': 20, 'was_impossible': False}),
 Prediction(uid='1958', iid='2926', r_ui=2.0, e

In [ ]:
import pandas as pd
df = pd.DataFrame(test_knn_pred)
df.rename(columns = {'uid': 'user_id','iid': 'item_id','r_ui':'og_rating','est':'pred_rating'}, inplace = True)

In [ ]:
df.drop(['details'], axis = 1)

,user_id,item_id,og_rating,pred_rating
0,4048,1839,3.0,2.671259
1,1181,2871,3.0,3.189054
2,451,2612,4.0,4.760734
3,673,2818,1.0,1.000000
4,4524,2699,3.0,3.163891
...,...,...,...,...
200037,2059,2717,2.0,2.436918
200038,343,2080,5.0,4.792007
200039,3868,2641,3.0,3.002048
200040,3243,165,4.0,3.778784


In [ ]:
precision, recall, f_score = precision_recall_fmeasure(mat_movies_users, 3.5)
ndcg_measure = ndcg(mat_movies_users)
print("The precision is ", precision)
print("The recall is ", recall)
print("The f-score is ", f_score)
print("The NDCG value is ", ndcg_measure)

The precision is  0.9
The recall is  0.58
The f-score is  0.7
The NDCG value is  0.97
